本notebook主要包括 以下部分：
- 神经网络迭代问题（P1）
该问题通过构建一个输入、隐藏、输出层数均为1层，节点数分别 为2，2，1的神经网络。通过编写代码计算的形式来确定不同迭代次数内的权重。
- 神经网络实操（P2）
通过两种方式分别构建了神经网络，
    1. 直接撰写代码，不调用库，实现一个输入层、隐藏层、输出层均为1层的简单神经网络（节点数：120,200,1），在第一个点位精度较高，但在后两个点位的精度并不很让人满意。
    2. 调用keras库，实现更复杂的神经网络，输入层、隐藏层、输出层数分别为1,2,1。节点数：120, 100, 10, 1。最终提升了精度。

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#  tqdm 为显示训练进度的库
from tqdm import tqdm
from tqdm._tqdm import trange
import matplotlib.pyplot as plt # plt 用于显示图片
import matplotlib.image as mpimg # mpimg 用于读取图片

# 1 神经网络迭代

神经网络的初始值和网络架构如下：

In [ ]:
pic = mpimg.imread('../input/pictures/p1.png') 
plt.imshow(pic) 
plt.axis('off')
plt.figure(dpi=150)
plt.show()

sigmoid为激活函数，deriv_sigmoid为激活函数的求导，mse为平均均方误差。

In [ ]:
# useful functions:
def sigmoid(z):
    # Sigmoid activation function: 
    # f(z) = 1 / (1 + e^(-z))
    return 1 / (1 + np.exp(-z))

def deriv_sigmoid(z):
    # Derivative of sigmoid: 
    # f'(x) = f(x) * (1 - f(x))
    return sigmoid(z) * (1 - sigmoid(z))

def mse(y_true, y_pred):
    # mean squared error
    return ((y_true - y_pred) ** 2).mean()

接下来搭建神经网络，搭建的网络为一个简单的2个输入层细胞，2个隐层细胞，1个输出细胞的网络。

该网络总共有6个权重系数，$w_1$ - $w_6$，3个偏置系数$b_1$- $b_3$，设置初始化均为0.1(按照例子里的初始化，也方便对答案)。

先定义前馈操作：

$o_1 = \sigma(w_1x_1+w_3x_2+b1)$

$o_2 = \sigma(w_2x_1+w_4x_2+b2)$

$o_3 = \sigma(w_5o_1+w_6o_2+b3)$

预测值为$o_3$，则其实

$\mathrm{Loss} = \frac{1}{N}\sum(o_3-y_{pred})^2$

第二步计算偏导，

根据PPT中的链式法则编写代码，其实链式法则主要分为三个部分，形同$o_i(1-o_i)$的部分，直接使用deriv_sigmoid(oi)处理，



$\frac{\partial L}{\partial w_5} = (o_3-y)\cdot o_3(1-o_3)\cdot o_1$

$\frac{\partial L}{\partial w_6} = (o_3-y)\cdot o_3(1-o_3)\cdot o_2$

$\frac{\partial L}{\partial b_3} = (o_3-y)\cdot o_3\cdot (1-o_3)$

$\frac{\partial L}{\partial w_1} = (o_3-y)\cdot o_3(1-o_3)\cdot w_5 \cdot o_1(1-o_1) \cdot x_1$

$\frac{\partial L}{\partial w_3} = (o_3-y)\cdot o_3(1-o_3)\cdot w_5 \cdot o_1(1-o_1) \cdot x_2$

$\frac{\partial L}{\partial b_1} = (o_3-y)\cdot o_3(1-o_3)\cdot w_5 \cdot o_1(1-o_1)$

$\frac{\partial L}{\partial w_2} = (o_3-y)\cdot o_3(1-o_3)\cdot w_6 \cdot o_2(1-o_2) \cdot x_1$

$\frac{\partial L}{\partial w_4} = (o_3-y)\cdot o_3(1-o_3)\cdot w_6 \cdot o_2(1-o_2) \cdot x_2$

$\frac{\partial L}{\partial b_2} = (o_3-y)\cdot o_3(1-o_3)\cdot w_6 \cdot o_2(1-o_2)$

In [ ]:
class NN():
    """
    A neural network with:
      - 2 inputs
      - a hidden layer with 2 neurons (o1, o2)
      - an output layer with 1 neuron (o3)
    """
    def __init__(self):
        # weights initializing
        self.w1 = 0.1
        self.w2 = 0.1
        self.w3 = 0.1
        self.w4 = 0.1
        self.w5 = 0.1
        self.w6 = 0.1
        # biases
        self.b1 = 0.1
        self.b2 = 0.1
        self.b3 = 0.1
        
    def feedforward(self, x):
        # feed the two sample to the NN work together
        o1 = sigmoid(self.w1 * x[:,0] + self.w3 * x[:,1] + self.b1)
        o2 = sigmoid(self.w2 * x[:,0] + self.w4 * x[:,1] + self.b2)
        o3 = sigmoid(self.w5 * o1 + self.w6 * o2 + self.b3)
        return o3
    
    def train(self, x, y_true):
        """
        - x is a (2 x 2) numpy array for 2 samples
        - y_true is a numpy array with 2 elements,representing to the training data
        """
        learn_rate = 0.1
        epochs = 2 # number of times to loop through the entire dataset
        
        for epoch in range(epochs):
                
            # Do a feedforward to get values
            h1 = self.w1 * x[:,0] + self.w3 * x[:,1] + self.b1
            o1 = sigmoid(h1)
            
            h2 = self.w2 * x[:,0] + self.w4 * x[:,1] + self.b2
            o2 = sigmoid(h2)
                
            h3 = self.w5 * o1 + self.w6 * o2 + self.b3
            o3 = sigmoid(h3)
            # o3 equals to y_pred
                
            # Calculate partial derivatives.
            # Naming principle: d_L_d_w1 represents "partial L / partial w1"
            d_L_d_o3 = o3-y_true
                
            # cell o3
            d_L_d_w5 = d_L_d_o3 * deriv_sigmoid(h3) * o1
            d_L_d_w6 = d_L_d_o3 * deriv_sigmoid(h3) * o2
            d_L_d_b3 = d_L_d_o3 * deriv_sigmoid(h3)
                
            # cell o1
            d_L_d_w1 = d_L_d_o3 * deriv_sigmoid(h3) * self.w5 * deriv_sigmoid(h1) * x[:,0]
            d_L_d_w3 = d_L_d_o3 * deriv_sigmoid(h3) * self.w5 * deriv_sigmoid(h1) * x[:,1]
            d_L_d_b1 = d_L_d_o3 * deriv_sigmoid(h3) * self.w5 * deriv_sigmoid(h1)
                
            # cell o2
            d_L_d_w2 = d_L_d_o3 * deriv_sigmoid(h3) * self.w6 * deriv_sigmoid(h2) * x[:,0]
            d_L_d_w4 = d_L_d_o3 * deriv_sigmoid(h3) * self.w6 * deriv_sigmoid(h2) * x[:,1]
            d_L_d_b2 = d_L_d_o3 * deriv_sigmoid(h3) * self.w6 * deriv_sigmoid(h2)
                
            # update weights and biases
            grad_unit = (sum(d_L_d_w1)**2 +sum(d_L_d_w2)**2 +sum(d_L_d_b1)**2 
                         +sum(d_L_d_w3)**2 +sum(d_L_d_w4)**2 +sum(d_L_d_b2)**2
                         +sum(d_L_d_w5)**2 +sum(d_L_d_w6)**2 +sum(d_L_d_b3)**2)
            
            # cell o3
            self.w5 -= learn_rate * sum(d_L_d_w5)/np.sqrt(grad_unit)
            self.w6 -= learn_rate * sum(d_L_d_w6)/np.sqrt(grad_unit)
            self.b3 -= learn_rate * sum(d_L_d_b3)/np.sqrt(grad_unit)
                
            # cell o1
            self.w1 -= learn_rate * sum(d_L_d_w1)/np.sqrt(grad_unit)
            self.w3 -= learn_rate * sum(d_L_d_w3)/np.sqrt(grad_unit)
            self.b1 -= learn_rate * sum(d_L_d_b1)/np.sqrt(grad_unit)
                
            # cell o2
            self.w2 -= learn_rate * sum(d_L_d_w2)/np.sqrt(grad_unit)
            self.w4 -= learn_rate * sum(d_L_d_w4)/np.sqrt(grad_unit)
            self.b2 -= learn_rate * sum(d_L_d_b2)/np.sqrt(grad_unit)

            y_pred = self.feedforward(x)
            loss = mse(y_true, y_pred)
            print("Epoch: {} loss: {}".format(epoch, loss))
            
            print("w1:{},w2:{},w3:{},\nw4:{},w5:{},w6:{},\nb1:{},b2:{},b3:{}".format(
                self.w1,self.w2,self.w3,self.w4,self.w5,self.w6,self.b1,self.b2,self.b3))
                
# dataset definition
x = np.array([
    [0.1, 0.1], 
    [0.2, 0.2],  
])
y_trues = np.array([
    0.3, # sample 1
    0.5, # sample 2
])

# Train
net = NN()
net.train(x, y_trues)

如上输出所示，第一次迭代结果：

w1: 0.1; w2: 0.1; w3:0.1; w4:0.1; w5:0.06; w6:0.06

b1:0.1; b2:0.1, b3:0.02

损失函数：0.02447(我用的是MSE，因此和excel中相差一半)

第二次迭代结果

w1: 0.1; w2: 0.1; w3:0.1; w4:0.1; w5:0.015; w6:0.015

b1:0.1;b2:0.1, b3:-0.06

损失函数：0.01792

# 2 神经网络实操

在此假设三个传感器点位相互独立，每次只用一个神经网络处理。于是，先处理第一个点位的数据

数据读取,并简单观察

In [ ]:
data = pd.read_excel('../input/nn-hw1-data/data_hw1.xlsx', sheet_name=0)
data.head()

先处理第一个传感器点位，即列索引为3.02的那一列。数据以天为单位，假设天与天之内独立。因此需要找到每天的样本点数

In [ ]:
data_np = np.array(data.iloc[:,1:])
# 查看每天的数据数
day_sampling_len = int(len(data)/31)
print(day_sampling_len)

由于我觉得，海浪数据应该有着明显的周期性质，因此，先查看一天内的数据，看一看内否找到周期性，周期性会影响样本的采样率，有效降低采样点，提高运算效率。

In [ ]:
# 第i天的数据
day_num = 5
day_samp = data[288*day_num:288*(day_num+1)]
day_samp.plot()

很可惜。。。并不能直观看到有什么周期性。因此尝试傅立叶变换，看是否能找到潜在的周期性。

In [ ]:
from scipy.fftpack import fft,ifft
from matplotlib.pylab import mpl
import heapq

mpl.rcParams['font.sans-serif'] = ['SimHei']   #显示中文
mpl.rcParams['axes.unicode_minus']=False       #显示负号

在送入FFT变换之后，尝试进行数据归一化，采用最简单的最大-最小值归一，将结果归一到0-1之间

In [ ]:
def normalize(x):
    max_val = x.max(axis=0, keepdims=True)
    min_val = x.min(axis=0, keepdims=True)
    return (x-min_val)/(max_val-min_val),max_val,min_val

day_samp_nor,max_,min_ = normalize(np.array(day_samp.iloc[:,1:]))

In [ ]:
len(day_samp_nor)

进行傅立叶变换

In [ ]:
fft_y=fft(day_samp_nor[:,1])
print(len(fft_y))
print(fft_y[0:5])

发现0附近特别大，这说明直流分量太大了= =。首先先去掉直流分量。

In [ ]:
fft_y[0] = 0

In [ ]:
N= 288
x = np.arange(N)           # 频率个数
 
abs_y=np.abs(fft_y)                # 取复数的绝对值，即复数的模(双边频谱)
angle_y=np.angle(fft_y)              #取复数的角度
 

In [ ]:
plt.figure()
plt.plot(x,abs_y)   
plt.title('Binary Spectrum')
 
plt.figure()
plt.plot(x,angle_y)   
plt.title('Binary Phase')
plt.show()

在这几张图已经可以明显的发现。。。可能周期性分析并没有什么用，提取最大的6个序列值

In [ ]:
n = 6
max_indexs = heapq.nlargest(n, range(len(abs_y)), abs_y.take)
print(max_indexs)

得到的结果，都是在直流范围附近的数据。这说明，序列可能不存在明显的周期性。于是，决定直接将预测点之前的所有120个样本点送入训练。

In [ ]:
import matplotlib.pyplot
%matplotlib inline
import imageio
import glob

同样定义神经网络。通过拓展之前写的代码，并参考了一些别人的工作，定义一个具有一个输入层、一个隐藏层、一个输出层的神经网络。变量如下：

- inodes: 输入节点数
- hnodes: 隐藏节点数
- onodes: 输出节点数 
- wih： 输入层到隐藏层的链接权重
- who： 隐藏层到输出层的链接权重
- activation_function: 激活函数

In [ ]:
np.random.seed(0)
# neural network with 3 layers
class MulitNN:
    # 神经网络初始化
    def __init__(self,inputnodes,hiddennodes,outputnodes,learningrate):
        # 定义网络 的输入节点数，隐藏层节点数，输出节点数
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        # learning rate
        self.lr = learningrate
        
        # 权重链接矩阵
        # 权重 w_i_j, 代表着从第i个节点到下一层第j个节点。
        # 比如： w12 w21 
        self.wih = (np.random.normal(0.0, pow(self.hnodes,-0.5), (self.hnodes,self.inodes) )  )
        self.who = (np.random.normal(0.0, pow(self.onodes,-0.5), (self.onodes,self.hnodes) )  )
        
        # 通过sigmoid函数激活
        # self.activation_function = lambda x: scipy.special.expit(x)
        self.activation_function = lambda x: sigmoid(x)
        pass
    
    # 网络训练
    def train(self,inputs_list,targets_list):      
        inputs = np.array(inputs_list,ndmin=2).T
        targets = np.array(targets_list,ndmin=2).T
        
        # 计算feedforward 
        # 计算输入到隐藏层，并激活
        hidden_inputs = np.dot(self.wih,inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # 计算隐藏层到输出层，并激活
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        # 输出层误差，也就是对损失函数对输出层的输入final_inputs的求导
        output_errors = final_outputs - targets
        # 隐藏层误差，用于计算损失函数对隐藏层权重的导数
        hidden_errors = np.dot(self.who.T, output_errors * final_outputs * (1.0 - final_outputs))
        
        # 更新who
        self.who -= self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
        
        # 更新wih
        self.wih -= self.lr * np.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        pass
    
    # 计算输出结果
    def predict(self,inputs_list):
        inputs = np.array(inputs_list,ndmin=2).T
        
        # 计算前向传递过程
        hidden_inputs = np.dot(self.wih,inputs)
        hidden_outputs = self.activation_function(hidden_inputs)
        final_inputs = np.dot(self.who, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

data.iloc用于访问第一列的数据，送入进行预测

In [ ]:
data_len = int(len(data)/288)-1
data_input,max_val,min_val= normalize(np.array(data.iloc[:,1:]))

对30天的处理数据进行处理，用于训练。第31天的数据用于预测

- x_list用于保存训练的数据x所组成的列表，

- y用于保存训练的目标数据

- value_index是选择的传感器点位（1表示选择第一个传感器点位）

- 找到序号为day_num的数据，提取前10小时共$10\times 12=120$个数据样本，作为输入数据，第121个数据作为训练数据

In [ ]:
x_list = []
y = []
value_index = 0
for day_num in tqdm(range(data_len)):
    # 找到序号为day_num的数据，提取120个数据样本,并找到value_index = 2的数据
    day_samp = data_input[288*day_num:(288*day_num+120)]
    day_value = day_samp[:,value_index]
    x_list.append(day_value)
    y.append(data_input[288*day_num+121,value_index])

In [ ]:
# 设置输入节点数为120，隐藏层200个节点，输出就1个点，建立神经网络
input_nodes = 120
hidden_nodes = 200
output_nodes = 1
# 学习率0.01
learning_rate = 0.001

nn_wave = MulitNN(input_nodes,hidden_nodes,output_nodes,learning_rate)

In [ ]:
# 训练1000次（反正训练的快）
epochs = 1000
for epoch in tqdm(range(epochs)):
    for i in range(len(x_list)):
        # 设置输入输出，进行训练
        inputs = x_list[i]
        target = y[i]
        # 训练
        nn_wave.train(inputs,target)
        pass
    output = nn_wave.predict(x_list)
    loss = np.mean((y-output)**2)
    pass

将第31天的数据送入系统，进行预测。

In [ ]:
pred_item = data_input[288*30:(288*30+120)]
output = nn_wave.predict(pred_item[:,0])

In [ ]:
output

预测结果通过反向归一化之后，得到原始的数据

In [ ]:
y_pred = output*(max_val[:,0]-min_val[:,0])+min_val[:,0]

In [ ]:
y_true = data_input[288*30+121,0]*(max_val[:,0]-min_val[:,0])+min_val[:,0]

In [ ]:
MAPE = abs(y_pred-y_true)/y_true
MAPE

查看预测结果，发现能达到2.45%左右的正确率？好像还挺高的

用同样的方法，预测其他两个传感器位的数据

In [ ]:
x2_list = []
y2 = []
# 设置不同的列 序号
value_index = 1
for day_num in tqdm(range(data_len)):
    # 找到序号为day_num的数据，提取120个数据样本,并找到value_index = 2的数据
    day_samp = data_input[288*day_num:(288*day_num+120)]
    day_value = day_samp[:,value_index]
    x2_list.append(day_value)
    y2.append(data_input[288*day_num+121,value_index])
    

In [ ]:
# 设置输入节点数为120，隐藏层200个节点，输出就1个点，建立神经网络
input_nodes = 120
hidden_nodes = 200
output_nodes = 1
# 学习率0.01
learning_rate = 0.001

nn_wave2 = MulitNN(input_nodes,hidden_nodes,output_nodes,learning_rate)
epochs = 1000
for epoch in tqdm(range(epochs)):
    for i in range(len(x_list)):
        inputs = x_list[i]
        target = y[i]
        nn_wave2.train(inputs,target)
        pass
    output = nn_wave2.predict(x_list)
    loss = np.mean((y-output)**2)
    pass

In [ ]:
pred_item = data_input[288*30:(288*30+120)]
output = nn_wave2.predict(pred_item[:,value_index])
y_pred = output*(max_val[:,value_index]-min_val[:,value_index])+min_val[:,value_index]
y_true = data_input[288*30+121,value_index]*(max_val[:,value_index]-min_val[:,value_index])+min_val[:,value_index]
MAPE = abs(y_pred-y_true)/y_true
MAPE

发现在第二个点位的误差就比较大了，为6.01%左右

In [ ]:
x3_list = []
y3 = []
value_index = 2
for day_num in tqdm(range(data_len)):
    # 找到序号为day_num的数据，提取120个数据样本,并找到value_index = 2的数据
    day_samp = data_input[288*day_num:(288*day_num+120)]
    day_value = day_samp[:,value_index]
    x3_list.append(day_value)
    y3.append(data_input[288*day_num+121,value_index])

In [ ]:
# 设置输入节点数为120，隐藏层200个节点，输出就1个点，建立神经网络
input_nodes = 120
hidden_nodes = 200
output_nodes = 1
# 学习率0.01
learning_rate = 0.001

nn_wave3 = MulitNN(input_nodes,hidden_nodes,output_nodes,learning_rate)

# 训练1000次（反正训练的快）
epochs = 1000
for epoch in tqdm(range(epochs)):
    for i in range(len(x_list)):
        record = x_list[i]
        inputs = record
        # create the target output values (all 0.01, except the desired label which is 0.99)
        target = y[i]
        # all_values[0] is the target label for this record
        nn_wave3.train(inputs,target)
        pass
    output = nn_wave3.predict(x_list)
    loss = np.mean((y-output)**2)
    pass

In [ ]:
pred_item = data_input[288*30:(288*30+120)]
output = nn_wave3.predict(pred_item[:,value_index])
y_pred = output*(max_val[:,value_index]-min_val[:,value_index])+min_val[:,value_index]
y_true = data_input[288*30+121,value_index]*(max_val[:,value_index]-min_val[:,value_index])+min_val[:,value_index]
MAPE = abs(y_pred-y_true)/y_true
MAPE

第三个点位的误差为8.49%左右，可以说是比较大的误差了。



## 方法1小结

综上，第一个点位准确率在0.7%,第二个点位6.01% ，第三个点位8.49%。这三个点位的数据预测精度中，第二个点位和第三个点位并不是很准确。这可能是因为模型略微简单导致。

接下来，尝试使用现有的keras神经网络库进行预测,看能不能在第三个点位获得更高的预测性能。

## Keras实现

为了看看能不能得到更好的结果，我使用Keras实现一个更加复杂的神经网络，看看效果。

In [ ]:
from keras.models import Model, load_model
from keras.layers import Dense, Activation, Input, Reshape, Dropout, Concatenate

In [ ]:
input_layer = Input(shape=(120,), name="input_data")
hidden1 = Dense(units=100, activation='tanh',name="hidden1")(input_layer)
hidden2 = Dense(units=10, name="hidden2")(hidden1)
output = Dense(units=1, name="output")(hidden2)

model = Model(inputs=input_layer,outputs=output)
model.compile(optimizer = 'Adam', loss='mse')

这个神经网络，为一个输入层，两个隐藏层，1个输出层。输入层120个节点，隐藏层1为100个节点，隐藏层2有10个节点。

In [ ]:
model.summary()

In [ ]:
x_list = []
y =  []

for day_num in tqdm(range(data_len+1)):
    # 找到序号为day_num的数据，提取120个数据样本,并找到value_index = 2的数据
    day_samp = data_input[288*day_num:(288*day_num+120),2]
    x_list.append(day_samp)
    y.append(data_input[288*day_num+121,2])

In [ ]:
x = np.array(x_list)
y = np.array(y)

In [ ]:
x_train = x[0:30]
x_test = x[30:]

In [ ]:
y_train = y[0:30]
y_test = y[30:]

In [ ]:
model.fit(x = x_train,y = y_train, batch_size = 2,epochs = 500)

In [ ]:
model.evaluate(x = x_test,y = y_test)

In [ ]:
pred_result = model.predict(x_test)

In [ ]:
y_pred = pred_result*(max_val[:,value_index]-min_val[:,value_index])+min_val[:,value_index]
y_true = y_test*(max_val[:,value_index]-min_val[:,value_index])+min_val[:,value_index]
MAPE = abs(y_pred-y_true)/y_true
MAPE

这次，预测精度达到了1.29%，相较于之前的8.5%,有了明显的提升。

## 方法2 小结

方法2通过keras来进行神经网络模型搭建。效果较为优良。原因可能和优化器的选择（Keras的模型使用了Adam的优化器）、模型复杂度等有关。

# 总结：

本文总共采用了两种方法构建神经网路。第一种参考了网上的一些资料，直接不调用库构建了一个1输入层（120个节点），1隐藏层（200个节点），1输出层的神经网络。但是效果并不是很完美。三个点位的预测误差分别为$2.45\%$, $6.01\%$, $8.49\%$

第二种，使用Keras构建了一个较为复杂的网络，其有1输入层（120），2隐藏层（节点为100，10），1输出层。这个神经网络预测的准确度相较简单的网络精度大大提高。最终使得第三个点位的预测精度达到了$1.29\%$

同时，因为没有发现数据的周期性，因此将输入时间点前120个数据点（0-10:00）都作为了输入，以预测10:05的数据。

### 3160100572 蒋柯越